In [222]:
# # webscraping_yahoo finance data

# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from bs4 import BeautifulSoup
# import pandas as pd
# import time

# chrome_options = Options()
# chrome_options.add_argument("--headless")

# driver = webdriver.Chrome(options=chrome_options)
# url = "https://finance.yahoo.com/quote/AAPL/history/?period1=345479400&period2=1761924776"

# driver.get(url)
# time.sleep(5)  # Wait for the dynamic table to load

# soup = BeautifulSoup(driver.page_source, "html.parser")

# # Correct table class selector based on your screenshot
# table = soup.find('table', {'class': 'table yf-1jecxey noDl hideOnPrint'})
# if table:
#     # Parse headers
#     headers = []
#     thead = table.find('thead')
#     if thead:
#         for th in thead.find_all('th'):
#             headers.append(th.text.strip())
#     # If headers list is empty, try checking the first tr's td
#     if not headers and thead:
#         first_tr = thead.find('tr')
#         if first_tr:
#             for td in first_tr.find_all('td'):
#                 headers.append(td.text.strip())

#     # Parse rows
#     rows = []
#     tbody = table.find('tbody')
#     for tr in tbody.find_all('tr'):
#         row = []
#         for td in tr.find_all('td'):
#             row.append(td.text.strip())
#         if row:
#             rows.append(row)

#     # Save to CSV
#     df = pd.DataFrame(rows, columns=headers if headers else None)
#     df.to_csv('yahoo_finance_AAPL_historical.csv', index=False)
#     print("CSV file saved as yahoo_finance_AAPL_historical.csv.")
# else:
#     print("Table not found.")

# driver.quit()

In [223]:
# # cleaning file

# import pandas as pd
# from datetime import datetime

# # Read the CSV file
# file_path = 'yahoo_finance_AAPL_historical.csv'
# df = pd.read_csv(file_path)

# # Remove rows that have 'Dividend' in any column
# df = df[~df.apply(lambda row: row.astype(str).str.contains('Dividend').any(), axis=1)]

# # Process each row
# def clean_row(row):
#     # Fix date format: "Oct 31, 2025" -> "Oct-31-2025"
#     try:
#         row[0] = datetime.strptime(row[0], "%b %d, %Y").strftime("%b-%d-%Y")
#     except Exception:
#         pass  # Skip rows where date is not in expected format
    
#     # Clean volume: "37,689,790" -> 37689790 (on last col)
#     try:
#         volume_str = str(row[-1]).replace(",", "")
#         if volume_str.isdigit():
#             row[-1] = int(volume_str)
#     except Exception:
#         pass  # If conversion fails, leave as is
#     return row

# df = df.apply(clean_row, axis=1)

# # Save the cleaned DataFrame back to the CSV file
# df.to_csv(file_path, index=False)
# print("Cleaned data saved to", file_path)


In [224]:
# data reading
import pandas as pd
from datetime import datetime

df = pd.read_csv('yahoo_finance_AAPL_historical.csv')

In [225]:
# checking data types
for col in df.columns:
    print(col, type(df[col].iloc[0]))

# null value handling
df.interpolate(method='linear', inplace=True)
null_counts = df.isnull().sum()
print(null_counts)

Date <class 'str'>
Open <class 'str'>
High <class 'numpy.float64'>
Low <class 'numpy.float64'>
Close <class 'numpy.float64'>
Adj_Close <class 'numpy.float64'>
Volume <class 'str'>
Label <class 'str'>
Predicted_Label <class 'str'>
Date               0
Open               0
High               0
Low                0
Close              0
Adj_Close          0
Volume             5
Label              0
Predicted_Label    0
dtype: int64


/tmp/ipykernel_113245/4144968491.py:6: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df.interpolate(method='linear', inplace=True)


In [226]:
# # Labelling data

# import pandas as pd

# csv_path = "/home/hp/Study/Code/AI/Equity_time_series_high_low_detector/yahoo_finance_AAPL_historical.csv"
# df = pd.read_csv(csv_path)

# df['Date'] = pd.to_datetime(df['Date'], format='%b-%d-%Y', errors='coerce')

# with open("reversal.txt", "r") as f:
#     reversal_dates = [line.strip() for line in f if line.strip()]

# df['Label'] = 'N'

# df.loc[df['Date'].dt.strftime('%Y-%m-%d').isin(reversal_dates), 'Label'] = 'R'

# df.to_csv(csv_path, index=False)

# print("✅ Labels updated successfully and date format normalized.")
# print(df.head())

In [230]:
# making chart.csv for graph plotting

import pandas as pd

# Assuming df already exists and has 'Date', 'Open', 'High', 'Low', 'Close' columns
path = "/home/hp/Study/Code/AI/Equity_time_series_high_low_detector/yahoo_finance_AAPL_historical.csv"
df = pd.read_csv(path)
start_date = "1981-06-27"
end_date = "2000-08-25"

df_filtered = df.copy()

# Ensure correct data types
df_filtered["Date"] = pd.to_datetime(df_filtered["Date"], errors="coerce")
numeric_cols = ["Open", "High", "Low", "Close"]
df_filtered[numeric_cols] = df_filtered[numeric_cols].apply(pd.to_numeric, errors="coerce")

# Drop rows with missing or invalid data
df_filtered = df_filtered.dropna(subset=["Date"] + numeric_cols)

# Filter by date range
df_filtered = df_filtered[(df_filtered["Date"] >= start_date) & (df_filtered["Date"] <= end_date)]

# Prepare and save data
df_to_save = df_filtered[["Date", "Open", "High", "Low", "Close","Label","Predicted_Label"]].copy()
df_to_save["Date"] = df_to_save["Date"].dt.strftime("%Y-%m-%d")
df_to_save.rename(columns={"Date": "time"}, inplace=True)

df_to_save.to_csv("chart.csv", index=False)

print(f"✅ Saved chart.csv successfully with columns: time, Open, High, Low, Close for date range {start_date} to {end_date}")

✅ Saved chart.csv successfully with columns: time, Open, High, Low, Close for date range 1981-06-27 to 2000-08-25


In [15]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score, precision_score, recall_score
import xgboost as xgb

file_path = "/home/hp/Study/Code/AI/Equity_time_series_high_low_detector/yahoo_finance_AAPL_historical.csv"
df = pd.read_csv(file_path)
df_original = df.copy()

df['Label'] = df['Label'].map({'N': 0, 'R': 1}).fillna(0).astype(int)
numeric_cols = ['Open', 'High', 'Low', 'Close', 'Adj_Close', 'Volume']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df = df.dropna(subset=numeric_cols).reset_index(drop=True)

X = df[numeric_cols].values
y = df['Label'].values
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

SEQ_LEN = 20
def create_sequences(data, labels, seq_len):
    X_seq, y_seq = [], []
    for i in range(len(data) - seq_len):
        X_seq.append(data[i:i+seq_len])
        y_seq.append(labels[i+seq_len])
    return np.array(X_seq), np.array(y_seq)

X_seq, y_seq = create_sequences(X_scaled, y, SEQ_LEN)
split_idx = int(len(X_seq) * 0.8)
X_train, y_train = X_seq[:split_idx], y_seq[:split_idx]
X_val, y_val = X_seq[split_idx:], y_seq[split_idx:]

class LSTMFeatureExtractor(nn.Module):
    def __init__(self, input_dim=6, hidden_dim=64, num_layers=3, feature_dim=64):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_dim, feature_dim)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]
        out = torch.relu(self.fc(out))
        return out

device = 'cuda' if torch.cuda.is_available() else 'cpu'
lstm_model = LSTMFeatureExtractor().to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([10.0], device=device))
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)

class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self): return len(self.X)
    def __getitem__(self, idx): return self.X[idx], self.y[idx]

train_loader = torch.utils.data.DataLoader(TimeSeriesDataset(X_train, y_train), batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(TimeSeriesDataset(X_val, y_val), batch_size=64, shuffle=False)

EPOCHS = 10
for epoch in range(EPOCHS):
    lstm_model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).unsqueeze(1)
        optimizer.zero_grad()
        feats = lstm_model(X_batch)
        pred = torch.sigmoid(torch.mean(feats, dim=1, keepdim=True))
        loss = criterion(pred, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{EPOCHS}] Loss: {total_loss/len(train_loader):.6f}")

lstm_model.eval()
with torch.no_grad():
    X_lstm_train = lstm_model(torch.tensor(X_train, dtype=torch.float32).to(device)).cpu().numpy()
    X_lstm_val = lstm_model(torch.tensor(X_val, dtype=torch.float32).to(device)).cpu().numpy()
    X_lstm_all = lstm_model(torch.tensor(X_seq, dtype=torch.float32).to(device)).cpu().numpy()

model = xgb.XGBClassifier(
    n_estimators=500,
    max_depth=8,
    learning_rate=0.02,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=(np.sum(y_train == 0) / np.sum(y_train == 1)),
    tree_method="hist",
    eval_metric="logloss",
    random_state=42
)

model.fit(X_lstm_train, y_train)

y_pred_train = model.predict(X_lstm_train)
y_pred_val = model.predict(X_lstm_val)

def print_metrics(y_true, y_pred, name):
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, zero_division=0)
    rec = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    cm = confusion_matrix(y_true, y_pred)
    print(f"\n{name} Metrics:")
    print(f"Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f}")
    print("Confusion Matrix:\n", cm)

print_metrics(y_train, y_pred_train, "Training (LSTM + XGBoost)")
print_metrics(y_val, y_pred_val, "Validation (LSTM + XGBoost)")

y_all_pred = model.predict(X_lstm_all)
pred_labels = np.where(y_all_pred == 1, 'R', 'N')

df_original = df_original.loc[SEQ_LEN:].copy().reset_index(drop=True)
df_original['Predicted_Label'] = pred_labels[:len(df_original)]
df_original.to_csv(file_path, index=False)
print("Done.")

Epoch [1/10] Loss: 1.127593
Epoch [2/10] Loss: 1.127177
Epoch [3/10] Loss: 1.127255
Epoch [4/10] Loss: 1.127333
Epoch [5/10] Loss: 1.127410
Epoch [6/10] Loss: 1.127255
Epoch [7/10] Loss: 1.127177
Epoch [8/10] Loss: 1.127333
Epoch [9/10] Loss: 1.127177
Epoch [10/10] Loss: 1.127410

Training (LSTM + XGBoost) Metrics:
Accuracy: 0.9593 | Precision: 0.0000 | Recall: 0.0000 | F1: 0.0000
Confusion Matrix:
 [[8586    0]
 [ 364    0]]

Validation (LSTM + XGBoost) Metrics:
Accuracy: 0.9781 | Precision: 0.0000 | Recall: 0.0000 | F1: 0.0000
Confusion Matrix:
 [[2189    0]
 [  49    0]]


ValueError: Length of values (11188) does not match length of index (11194)